In [9]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import time

In [10]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import time

def score_tickers(tickers, lookback_days=365):
    
    end_date = datetime.today()
    start_date = end_date - timedelta(days=lookback_days*2)  # extra buffer for momentum
    price_data = yf.download(tickers, start=start_date, end=end_date, progress=False)["Close"]
    price_data = price_data.dropna(thresh=len(price_data)*0.8, axis=1)
    valid_tickers = price_data.columns.tolist()
    
    if not valid_tickers:
        raise ValueError("No valid tickers with sufficient price data!")

    factors = pd.DataFrame(index=valid_tickers)
    
    #Momentum
    if len(price_data) > lookback_days:
        factors["momentum"] = (price_data.iloc[-1] / price_data.iloc[-lookback_days]) - 1
    else:
        factors["momentum"] = np.nan
    
    #Volatility
    daily_returns = price_data.pct_change().dropna()
    factors["volatility"] = daily_returns.std()
    
    #Fundamental factors
    value_dict = {}
    quality_dict = {}
    ebitda_margin_dict = {}
    operating_margin_dict = {}
    debt_ratio_dict = {}
    free_cash_flow_dict = {}

    for t in valid_tickers:
        try:
            info = yf.Ticker(t).info

            pb = info.get("priceToBook", np.nan)
            value_dict[t] = 1/pb if pb and pb > 0 else 0

            roe = info.get("returnOnEquity", np.nan)
            quality_dict[t] = roe if roe else 0

            ebitda = info.get("ebitdaMargins", np.nan)
            ebitda_margin_dict[t] = ebitda if ebitda else 0

            op_margin = info.get("operatingMargins", np.nan)
            operating_margin_dict[t] = op_margin if op_margin else 0

            debt_eq = info.get("debtToEquity", np.nan)
            debt_ratio_dict[t] = -debt_eq if debt_eq and debt_eq > 0 else 0

            fcf = info.get("freeCashflow", np.nan)
            free_cash_flow_dict[t] = fcf/1e9 if fcf else 0

        except:
            value_dict[t] = 0
            quality_dict[t] = 0
            ebitda_margin_dict[t] = 0
            operating_margin_dict[t] = 0
            debt_ratio_dict[t] = 0
            free_cash_flow_dict[t] = 0

        time.sleep(0.2)  # reduce Yahoo API throttling risk

    # Assign fundamental factors to DataFrame
    factors["value"] = pd.Series(value_dict).reindex(factors.index).fillna(0)
    factors["quality"] = pd.Series(quality_dict).reindex(factors.index).fillna(0)
    factors["ebitda_margin"] = pd.Series(ebitda_margin_dict).reindex(factors.index).fillna(0)
    factors["operating_margin"] = pd.Series(operating_margin_dict).reindex(factors.index).fillna(0)
    factors["debt_ratio"] = pd.Series(debt_ratio_dict).reindex(factors.index).fillna(0)
    factors["free_cash_flow"] = pd.Series(free_cash_flow_dict).reindex(factors.index).fillna(0)
    factors["catalyst"] = 0
    
    # Normalize factors
    for col in ["momentum", "value", "quality", "volatility",
                "ebitda_margin", "operating_margin", "debt_ratio", "free_cash_flow"]:
        if factors[col].std() != 0:
            factors[col] = (factors[col] - factors[col].mean()) / factors[col].std()
        else:
            factors[col] = 0

    # Lower volatility preferred
    factors["volatility"] = -factors["volatility"]

    # Q_score calculation
    weights = {
        "momentum": 0.20,
        "value": 0.15,
        "quality": 0.20,
        "volatility": 0.10,
        "ebitda_margin": 0.10,
        "operating_margin": 0.10,
        "debt_ratio": 0.10,
        "free_cash_flow": 0.10,
        "catalyst": 0.05
    }
    factors["Q_score"] = sum(factors[f] * w for f, w in weights.items())
    
    return factors.sort_values("Q_score", ascending=False).round(3)



In [11]:
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
factors_df = score_tickers(tickers)
print(factors_df)

/var/folders/vb/m9plnt0538gc2wb4l5pzcgr80000gp/T/ipykernel_44349/3723994972.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data = yf.download(tickers, start=start_date, end=end_date, progress=False)["Close"]


       momentum  volatility  value  quality  ebitda_margin  operating_margin  \
MSFT     -0.558       0.786  0.101   -0.298          1.397             1.228   
GOOGL    -0.130       0.293  0.645   -0.271          0.347             0.475   
AAPL     -0.242       0.422 -1.334    1.757          0.154             0.327   
AMZN     -0.796       0.249  1.194   -0.448         -0.722            -0.794   
TSLA      1.726      -1.749 -0.607   -0.741         -1.176            -1.237   

       debt_ratio  free_cash_flow  catalyst  Q_score  
MSFT        0.341           0.387         0    0.258  
GOOGL       0.702           0.062         0    0.204  
AAPL       -1.735           1.359         0    0.156  
AMZN        0.083          -0.478         0   -0.236  
TSLA        0.611          -1.330         0   -0.382  


PORTFOLIO OPTIMIZER - MAXIMIZES WEIGHT AT 25% OF PORTFOLIO

In [ ]:
from scipy.optimize import minimize


factors = factors_df   # CSV with tickers and Q_scores
risk_free_rate = 0.03                     # risk-free rate for Sharpe
max_weight = 0.25                          # max weight per ticker



tickers = factors.index.tolist()
Q_scores = factors["Q_score"].values

# Map Q_scores to expected annual returns
expected_returns = 0.08 + 0.12 * (Q_scores - Q_scores.mean()) / Q_scores.std()


end_date = datetime.today()
start_date = end_date - timedelta(days=365*3)

price_data = yf.download(tickers, start=start_date, end=end_date, progress=False)["Close"]
price_data = price_data.dropna(thresh=len(price_data)*0.8, axis=1)

daily_returns = price_data.pct_change().dropna()
cov_matrix = daily_returns.cov() * 252  # annualized covariance


def portfolio_volatility(weights, cov_matrix):
    return np.sqrt(weights.T @ cov_matrix.values @ weights)

def portfolio_return(weights, expected_returns):
    return np.dot(weights, expected_returns)

def neg_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free):
    port_return = portfolio_return(weights, expected_returns)
    port_vol = portfolio_volatility(weights, cov_matrix)
    return -(port_return - risk_free) / port_vol  # negative because we minimize


n = len(tickers)
x0 = np.array([1/n]*n)
bounds = [(0, max_weight) for _ in range(n)]
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

result = minimize(neg_sharpe_ratio, x0, args=(expected_returns, cov_matrix, risk_free_rate),
                  method='SLSQP', bounds=bounds, constraints=constraints)

optimal_weights = pd.Series(result.x, index=tickers)
optimal_weights = optimal_weights.sort_values(ascending=False)


print("Optimal Portfolio Weights (Sharpe Max):")
print(optimal_weights)
print(f"\nExpected Portfolio Return: {portfolio_return(result.x, expected_returns):.4f}")
print(f"Expected Portfolio Volatility: {portfolio_volatility(result.x, cov_matrix):.4f}")
sharpe = (portfolio_return(result.x, expected_returns) - risk_free_rate) / portfolio_volatility(result.x, cov_matrix)
print(f"Expected Sharpe Ratio: {sharpe:.4f}")

/var/folders/vb/m9plnt0538gc2wb4l5pzcgr80000gp/T/ipykernel_44349/2736043704.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data = yf.download(tickers, start=start_date, end=end_date, progress=False)["Close"]


Optimal Portfolio Weights (Sharpe Max):
GOOGL    2.500000e-01
AAPL     2.500000e-01
AMZN     2.500000e-01
MSFT     2.500000e-01
TSLA     1.665335e-16
dtype: float64

Expected Portfolio Return: 0.1243
Expected Portfolio Volatility: 0.2406
Expected Sharpe Ratio: 0.3920
